In [46]:
import pandas as pd
import datetime
import numpy as np
import datetime
from sklearn.linear_model import LogisticRegression
epoch = datetime.datetime.utcfromtimestamp(0)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data_train_partial.csv')


In [4]:
def multiply(row):
    return row['x_entry']*row['y_entry']

In [5]:
multxy = np.zeros(len(df))
df['multxy'] = multxy

In [6]:
df['multxy'] = df.apply(multiply, axis=1)

In [7]:
df_clean = df.dropna(subset=['time_entry','time_exit','x_entry','y_entry'])

In [8]:
X = df.iloc[:, [0,1,2,3,5]].values

In [9]:
y = df['target'].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [11]:
resp = np.bincount(y_test)
ii = np.nonzero(resp)[0]
np.vstack((ii,resp[ii])).T

array([[    0, 28688],
       [    1, 24186]])

In [12]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [18]:
classifier = RandomForestClassifier(random_state=42, max_depth=20)
classifier.fit(X_train_std, y_train)

/userhome/comp3314/c3314g15/anaconda3/envs/tf36/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [22]:
y_predicted= classifier.predict(X_test_std)
accuracy_score(y_test, y_predicted)

0.919241971479366

In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)
cm

array([[27118,  1570],
       [ 2700, 21486]])

# PreProcessing Test Data

In [24]:
df_test = pd.read_csv('data_test.csv', parse_dates=['time_exit', 'time_entry'])

In [42]:
df_test = df_test.drop_duplicates(subset='hash', keep='last')
len(df_test)

33515

In [50]:
def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [51]:
df_test_new_entry = df_test['time_entry'].apply(unix_time_millis)
df_test_new_exit = df_test['time_exit'].apply(unix_time_millis)

df_test['time_entry'] = df_test_new_entry
df_test['time_exit'] = df_test_new_exit

df_test['multxy'] = df.apply(multiply, axis=1)

TypeError: unsupported operand type(s) for -: 'float' and 'datetime.datetime'

In [49]:
df_test_clean = df_test

In [34]:
X_test = df_test_clean.iloc[:, [3,4,8,9,12]].values


In [37]:
def classify_city_center(row):
    if (row['x_exit'] > 3750901.5068) and  (row['x_exit'] < 3770901.5068) and (row['y_exit'] > -19268905.6133) and (row['y_exit'] < -19208905.6133):
        return 1
    else:
        return 0
y_pred = df_test_clean.apply(classify_city_center, axis=1)

In [39]:
df_final = df_test_clean.iloc[:, [2]]
df_final['target'] = y_pred
df_final.columns=['id','target']
df_final.describe()

/userhome/comp3314/c3314g15/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,target
count,202937.000000
mean,0.253433
std,0.434978
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000
